In [ ]:
# Import matplotlib before seaborn
import matplotlib as mpl
import matplotlib.pyplot as plt
import itertools  # for color palette cycling
import re
import pandas as pd
import seaborn as sns
import sys
from cycler import cycler
import seaborn as sns
%matplotlib inline

In [ ]:
from datetime import datetime
import pickle
import timeit

import networkx as nx

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
sys.version_info.major

In [ ]:
assert sys.version_info.major == 3

In [ ]:
import networkx_helpers as nxh
import networkx_explore as nxe

In [ ]:
! pwd

In [ ]:
make_again = False

edges_path = '/work/rnaseq/pcor_new/Ledoit_Wolf_r4.8xlarge_244GB_memory/ledoit_wolf_precision_cutoff_0.005--26967_genes.tsv'
gff_tsv_path = '/work/m4b_binning/assembly/prokka/contigs/contigs_longer_than_1500bp/contigs_longer_than_1500bp_gffs_concatenated.gff.genes.tsv'
tail_percent = 2.5

if make_again:
    network = nxh.build_network(edges_path, tail_percent, gff_tsv_path)
    date = datetime.today().strftime('%y%m%d')
    print(date)
    networkx.write_gpickle(network, date + '_ledoit_wolf_26967_genes_2.5percent_network.gpickle')
else:
    # favorite_color = pickle.load( open( "save.p", "rb" ) )
    p_path = './170401ledoit_wolf_26967_genes_2.5percent_network.gpickle'
    network = pickle.load(open(p_path, 'rb'))

In [ ]:
edges = nxh.load_edges(edges_path, tail_percent)
edges.head()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,3))
edges['pcor'].hist(bins=100, ax=ax)
ax.set_yscale('log')
ax.get_xaxis().set_major_formatter(
    mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

sns.despine()
locs, labels = plt.xticks()
plt.setp(labels, rotation=45)


In [ ]:
degree_sequence=sorted(nx.degree(network).values(),reverse=True) # degree sequence

In [ ]:
len(degree_sequence)

In [ ]:
! mkdir -p results

In [ ]:
print(nx.info(network))

In [ ]:
sns.set_style("white")
p = nxe.plot_degree_rank(network)
p.savefig('./results/170401_degree_rank_plot--before_pcor_problem_was_discovered.pdf',
          bbox_inches='tight')

In [ ]:
# takes > 50 min and doesn't complete
# networkx.draw_networkx(network)

In [ ]:
# get_node_attributes(G, name)
pcors_dict = nx.get_edge_attributes(network, 'pcor')

dict(list(pcors_dict.items())[0:5])

In [ ]:
# Warning: some of these edges are missing gene labels. (??)
low_conn = [k for k, v in nx.degree(network).items() if v < 12]
print(len(low_conn))
low_conn

In [ ]:
network['5_168418']

In [ ]:
# get_node_attributes(G, name)
names_dict = nx.get_node_attributes(network, 'product')
print(len(names_dict))
# print first few items
dict(list(names_dict.items())[0:5])

In [ ]:
names_dict['1_50186']

In [ ]:
import networkx_explore as nxe

In [ ]:
nodes_list = ['1_50186', '1_73594', '1_94876']
nxe.draw(nxe.graph_by_nodes_list(network, nodes_list))

In [ ]:
pmmos1 = ['1_66816', '1_66817', '1_66818']
nxe.draw(nxe.graph_by_nodes_list(network, pmmos1))

In [ ]:
pmmos2 = ['3_138947', '3_138948', '3_138949'] 
nxe.draw(nxe.graph_by_nodes_list(network, pmmos2))

In [ ]:
pmmos3 = ['4_79604', '4_79605', '4_79606'] 
nxe.draw(nxe.graph_by_nodes_list(network, pmmos2))

In [ ]:
keys = []
for key, value in names_dict.items():
    if 'ethane' in value: 
        keys.append(key)
keys = sorted(keys)
for key in keys:
    print(key, names_dict[key])

In [ ]:
nxe.draw(nxe.graph_by_nodes_list(network, keys), nx.spring_layout)

In [ ]:
print(names_dict['2_20547'])

In [ ]:
low_conn = [k for k, v in nx.degree(network).items() if v < 10]
print(len(low_conn))
low_conn

In [ ]:
# ?? 
#print(names_dict['5_168418'])

In [ ]:
#print(names_dict['5_63560'])
network['5_63560']
#network['5_63560']

In [ ]:
a = [1,2,3,4]
print(list(itertools.combinations(a,2)))

In [ ]:
#mmo_list = ['A', 'B', 'C'] 
mmo_list = ['3_138947', '3_138948', '3_138949'] 
combos = list(itertools.combinations(mmo_list, 2))
#combos = [(x, y) for (x, y) in combos if x != y]
combos

In [ ]:
sys.path.append('/work/rnaseq/pcor_new')
import network_prep

In [ ]:
print(network_prep.RAW_DATA_PATH)

In [ ]:
reads_path = network_prep.RAW_DATA_PATH
locus_regex = network_prep.LOCUS_REGEX
read_counts = pd.read_csv(reads_path, sep='\t')
read_counts['id'] = read_counts['locus'].str.extract(locus_regex, expand=True)
del read_counts['locus']
read_counts.head(3)

In [ ]:
read_counts = read_counts.set_index('id').T
read_counts.head()

In [ ]:
read_counts.columns[0:10]

In [ ]:
from textwrap import wrap

def plot_expression_scatters_by_pair(nodes_list):
    subplot_vars = list(itertools.combinations(nodes_list, 2))
    print(subplot_vars)
    fig, axs = plt.subplots(len(subplot_vars), 1, 
                            figsize = (4, 7),
                            sharex=True, sharey=True)
    print(axs)
    for axnum, ax in enumerate(fig.axes):
        s1_var, s2_var = subplot_vars[axnum]
        s1 = read_counts[s1_var]
        s2 = read_counts[s2_var]
        if s1.max() <= s2.max():
            print('swtich x and y series so the larger magnitude one is x')
            xvar = s2_var
            yvar = s1_var
            x = s2
            y = s1
        else:
            print("don't swtich x and y series")
            xvar = s1_var
            yvar = s2_var
            x = s1
            y = s2
            
        ax.plot(x, y, marker='o', linestyle='', alpha = 0.5)
        
        xname = '\n'.join(wrap(names_dict[xvar], 50))
        yname = '\n'.join(wrap(names_dict[yvar], 22))
        ax.set_xlabel(xname)
        ax.set_ylabel(yname)
        
        ax.get_xaxis().set_major_formatter(
            mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
        plt.xticks(rotation=90)
        
        ax.get_yaxis().set_major_formatter(
            mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
        
        for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
            item.set_fontsize(9)
        plt.subplots_adjust(hspace=0.5)
    return fig
    
mmo_list = ['3_138947', '3_138948', '3_138949'] 
p = plot_expression_scatters_by_pair(mmo_list)
    

In [ ]:
p = plot_expression_scatters_by_pair(pmmos1)
p.savefig('./results/170402_correlations_between_pMMO_subunits--set1.pdf',
          bbox_inches='tight')

In [ ]:
p = plot_expression_scatters_by_pair(pmmos2)
p.savefig('./results/170402_correlations_between_pMMO_subunits--set2.pdf',
          bbox_inches='tight')

In [ ]:
p = plot_expression_scatters_by_pair(pmmos3)
p.savefig('./results/170402_correlations_between_pMMO_subunits--set3.pdf',
          bbox_inches='tight')

In [ ]:
! realpath results/

In [ ]:
# edges_path = '/work/rnaseq/pcor_new/Ledoit_Wolf_r4.8xlarge_244GB_memory/ledoit_wolf_precision_cutoff_0.005--26967_genes.tsv'
# gff_tsv_path = '/work/m4b_binning/assembly/prokka/contigs/contigs_longer_than_1500bp/contigs_longer_than_1500bp_gffs_concatenated.gff.genes.tsv'

edges_df = nxh.load_edges(edges_path, 2.5)
edges_df.head()

In [ ]:
edges_df.head()

In [ ]:
pmmo_sets = pmmos3 + pmmos2 + pmmos1
pmmo_sets

In [ ]:
edges_df[edges_df['gene A'].isin(pmmo_sets) &
         edges_df['gene B'].isin(pmmo_sets)].sort_values(
    ['gene A', 'gene B'])

In [ ]:
read_counts.head(3)

In [ ]:
pmmo_counts = \
    read_counts[pmmo_sets]
pmmo_counts.head(3)

In [ ]:
from sklearn import covariance

In [ ]:
lw = covariance.LedoitWolf().fit(pmmo_counts.as_matrix())

In [ ]:
sns.heatmap(lw.get_precision())

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
for r in range(1,9):
    print(r)

In [ ]:
! pwd

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,5))
ss = StandardScaler(with_mean=False, with_std=True)
scaled_mmo = ss.fit_transform(pmmo_counts)  #shape [n_samples, n_features]
lw_ss = covariance.LedoitWolf(assume_centered=True).fit(scaled_mmo)
#sns.heatmap(scaled_mmo)
plot_df = pd.DataFrame(lw_ss.get_precision(), 
                       columns=range(1,9+1), index=range(1,9+1))
sns.heatmap(plot_df, ax=ax,
            cbar_kws={'label': 'partial\ncorrelation'}, annot=False)
ax.set_xlabel('pMMO gene')
ax.set_ylabel('pMMO gene')
fig.savefig('./results/170403_pMMO_pcor_heatmap.pdf',
           bbox_inches='tight')

In [ ]:
pmmos3_counts = read_counts[pmmos3]
#print(pmmos3_counts.head(20))

ss = StandardScaler(with_mean=False, with_std=True)
scaled_pmmos3_counts = ss.fit_transform(pmmos3_counts)
print('scaled_pmmos3_counts:')
#print(scaled_pmmos3_counts.head(10))

lw_ss_pmmos3_counts = covariance.LedoitWolf().fit(scaled_pmmos3_counts)
#sns.heatmap(scaled_mmo)
#sns.heatmap(lw_ss_pmmos3_counts.get_precision())
sns.heatmap(lw_ss_pmmos3_counts.precision_)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (5, 2))
pmmos3_counts['4_79604'].hist(ax=ax)
ax.set_xlabel('counts')
ax.set_ylabel('# of samples')

In [ ]:
import numpy as np

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(8, 8),
                       sharex=False, sharey=True)
axl = np.ravel(axs)
plot_num = 0
for p in pmmo_sets:
    ax = axl[plot_num]
    plot_num += 1
    ax.set_title(p)
    ax.set_ylabel('# of samples')
    #print(pmmo_counts[p])
    pmmo_counts[p].hist(ax=ax, bins=20)
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)
    ax.get_xaxis().set_major_formatter(
        mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
plt.subplots_adjust(wspace=.4, hspace=.65)
fig.savefig('./results/170403_read_count_distributions--3_pmoCAB_clusters.pdf',
            bbox_inches='tight')


In [ ]:
pmmo_counts.to_csv('./results/sample_pMMO_counts.tsv', 
                   sep='\t')

In [ ]:
pmmo_counts.head(2)

In [ ]:
read_counts.shape

In [ ]:
pmmo_sets

In [ ]:
keys = []
for key, value in names_dict.items():
    if 'ethanol dehydrogenase' in value: 
        keys.append(key)
keys = sorted(keys)
for key in keys:
    print(key, names_dict[key])

In [ ]:
keys = []
for key, value in names_dict.items():
    if '3-hexulose-6-phosphate' in value: 
        keys.append(key)
keys = sorted(keys)
for key in keys:
    print(key, names_dict[key])

In [ ]:
hps_hpi_pairs = [
    ['1_142178', '1_142179'], # no pcor in this set
    ['1_148469', '1_148470'], # pcor = - 31.46
    ['2_159112', '2_159113'], # pcor = - 4.3
    ['4_43633', '4_43634'],   # pcor = - 133.96
    ['5_111291', '5_111292']  # no pcor in this set. 
    ]

In [ ]:
# nxe.draw(nxe.graph_by_nodes_list(network, pmmos2))
nxe.draw(nxe.graph_by_nodes_list(network, hps_hpi_pairs[4]))

In [ ]:
for h in hps_hpi_pairs:
    print(h)
    print('---')
    nxe.draw(nxe.graph_by_nodes_list(network, h))

In [ ]:
from textwrap import wrap

def plot_expression_scatters_by_pair(nodes_list):
    subplot_vars = list(itertools.combinations(nodes_list, 2))
    print(subplot_vars)
    fig, axs = plt.subplots(len(subplot_vars), 1, 
                            figsize = (4, 7),
                            sharex=True, sharey=True)
    print(axs)
    for axnum, ax in enumerate(fig.axes):
        s1_var, s2_var = subplot_vars[axnum]
        s1 = read_counts[s1_var]
        s2 = read_counts[s2_var]
        if s1.max() <= s2.max():
            print('swtich x and y series so the larger magnitude one is x')
            xvar = s2_var
            yvar = s1_var
            x = s2
            y = s1
        else:
            print("don't swtich x and y series")
            xvar = s1_var
            yvar = s2_var
            x = s1
            y = s2
            
        ax.plot(x, y, marker='o', linestyle='', alpha = 0.5)
        
        xname = '\n'.join(wrap(names_dict[xvar], 50))
        yname = '\n'.join(wrap(names_dict[yvar], 22))
        ax.set_xlabel(xname)
        ax.set_ylabel(yname)
        
        ax.get_xaxis().set_major_formatter(
            mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
        plt.xticks(rotation=90)
        
        ax.get_yaxis().set_major_formatter(
            mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
        
        for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
            item.set_fontsize(9)
        plt.subplots_adjust(hspace=0.5)
    return fig
    
mmo_list = ['3_138947', '3_138948', '3_138949'] 
p = plot_expression_scatters_by_pair(mmo_list)
    

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(8, 8),
                       sharex=False, sharey=True)
axl = np.ravel(axs)
plot_num = 0
for p in pmmo_sets:
    ax = axl[plot_num]
    plot_num += 1
    ax.set_title(p)
    ax.set_ylabel('# of samples')
    #print(pmmo_counts[p])
    pmmo_counts[p].hist(ax=ax, bins=20)
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)
    ax.get_xaxis().set_major_formatter(
        mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
plt.subplots_adjust(wspace=.4, hspace=.65)
fig.savefig('./results/170403_read_count_distributions--3_pmoCAB_clusters.pdf',
            bbox_inches='tight')
